In [3]:
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Lambda
from keras.layers import Embedding
from keras.layers import Convolution1D,MaxPooling1D, Flatten
from keras.datasets import imdb
from keras import backend as K
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.utils.np_utils import to_categorical

from sklearn.preprocessing import Normalizer
from keras.models import Sequential
from keras.layers import Convolution1D, Dense, Dropout, Flatten, MaxPooling1D
from keras.utils import np_utils
import numpy as np
import h5py
from keras import callbacks
from keras.layers import LSTM, GRU, SimpleRNN, Bidirectional
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

traindata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_training-set.csv')
testdata = pd.read_csv('/home/ritika/NetworkIntrusionDetection/UNSW-NB15 - CSV Files/a part of training and testing set/UNSW_NB15_testing-set.csv')

traindata=traindata.drop('id',axis=1)
testdata=testdata.drop('id',axis=1)

from sklearn.preprocessing import LabelEncoder
lb_make = LabelEncoder()

traindata['proto'] = lb_make.fit_transform(traindata['proto'])
traindata['service'] = lb_make.fit_transform(traindata['service'])
traindata['state'] = lb_make.fit_transform(traindata['state'])

testdata['proto'] = lb_make.fit_transform(testdata['proto'])
testdata['service'] = lb_make.fit_transform(testdata['service'])
testdata['state'] = lb_make.fit_transform(testdata['state'])

X = traindata.iloc[:,1:42]
Y = traindata.iloc[:,-1]
x = testdata.iloc[:,1:42]
y = testdata.iloc[:,-1]

In [4]:
Arr = X.values
arr = x.values

import librosa
Arr_Delta = librosa.feature.delta(Arr)
arr_delta = librosa.feature.delta(arr)

Delta1 = pd.DataFrame(Arr_Delta)
delta1 = pd.DataFrame(arr_delta)

scaler = Normalizer().fit(Delta1)
Delta1 = scaler.transform(Delta1)
scaler = Normalizer().fit(delta1)
delta1 = scaler.transform(delta1)

Delta1 = np.reshape(Delta1, (Delta1.shape[0],Delta1.shape[1],1))
delta1 = np.reshape(delta1, (delta1.shape[0],delta1.shape[1],1))

scaler = Normalizer().fit(X)
X = scaler.transform(X)
scaler = Normalizer().fit(x)
x = scaler.transform(x)

Y = np.array(Y)
y = np.array(y)

X = np.reshape(X, (X.shape[0],X.shape[1],1))
x = np.reshape(x, (x.shape[0],x.shape[1],1))

lstm_output_size = 70

In [5]:
from keras.layers import Input, Convolution1D, Dense, Dropout, Flatten, MaxPooling1D, concatenate, Reshape
from keras.models import Model

lstm_output_size=70
Xshape = Input(shape=(41, 1))
Delta1shape = Input(shape=(41, 1))

x1 = Convolution1D(64, 3, border_mode="same",activation="relu")(Xshape)
x1 = MaxPooling1D(pool_length=(2))(x1)
x1 = Dropout(0.5)(x1)
   
x2 = Convolution1D(64, 3, border_mode="same",activation="relu")(Delta1shape)
x2 = MaxPooling1D(pool_length=(2))(x2)
x2 = Dropout(0.5)(x2)

output = concatenate([x1, x2])

x3 = Bidirectional(LSTM(lstm_output_size))(output)
x3 = Dropout(0.1)(x3)
x3 = Dense(1, activation="sigmoid")(x3)

model = Model([Xshape, Delta1shape],x3)
model.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
print(model.summary())

/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, activation="relu", padding="same")`
  if sys.path[0] == '':
/home/ritika/.virtualenvs/pythree/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 41, 1)        0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 41, 64)       256         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 41, 64)       256         input_2[0][0]                   

In [6]:
history = model.fit([X,Delta1], Y, epochs=200, validation_data=([x,delta1], y))


Train on 175341 samples, validate on 82332 samples
Epoch 1/200
175341/175341 [==============================] - 173s 986us/step - loss: 0.4132 - accuracy: 0.7683 - val_loss: 0.5572 - val_accuracy: 0.6862
Epoch 2/200
175341/175341 [==============================] - 171s 977us/step - loss: 0.3736 - accuracy: 0.8153 - val_loss: 0.5397 - val_accuracy: 0.6890
Epoch 3/200
175341/175341 [==============================] - 169s 962us/step - loss: 0.3643 - accuracy: 0.8196 - val_loss: 0.5604 - val_accuracy: 0.6893
Epoch 4/200
175341/175341 [==============================] - 317s 2ms/step - loss: 0.3611 - accuracy: 0.8234 - val_loss: 0.5556 - val_accuracy: 0.6954
Epoch 5/200
175341/175341 [==============================] - 322s 2ms/step - loss: 0.3600 - accuracy: 0.8235 - val_loss: 0.5442 - val_accuracy: 0.7231
Epoch 6/200
175341/175341 [==============================] - 323s 2ms/step - loss: 0.3572 - accuracy: 0.8260 - val_loss: 0.5511 - val_accuracy: 0.7275
Epoch 7/200
175341/175341 [=========

175341/175341 [==============================] - 308s 2ms/step - loss: 0.2794 - accuracy: 0.8740 - val_loss: 0.4992 - val_accuracy: 0.7351
Epoch 54/200
175341/175341 [==============================] - 311s 2ms/step - loss: 0.2790 - accuracy: 0.8740 - val_loss: 0.4994 - val_accuracy: 0.7351
Epoch 55/200
175341/175341 [==============================] - 306s 2ms/step - loss: 0.2785 - accuracy: 0.8746 - val_loss: 0.5030 - val_accuracy: 0.7347
Epoch 56/200
175341/175341 [==============================] - 307s 2ms/step - loss: 0.2784 - accuracy: 0.8745 - val_loss: 0.5012 - val_accuracy: 0.7337
Epoch 57/200
175341/175341 [==============================] - 307s 2ms/step - loss: 0.2772 - accuracy: 0.8748 - val_loss: 0.5235 - val_accuracy: 0.7356
Epoch 58/200
175341/175341 [==============================] - 309s 2ms/step - loss: 0.2784 - accuracy: 0.8739 - val_loss: 0.5091 - val_accuracy: 0.7349
Epoch 59/200
175341/175341 [==============================] - 309s 2ms/step - loss: 0.2778 - accuracy

175341/175341 [==============================] - 309s 2ms/step - loss: 0.2667 - accuracy: 0.8770 - val_loss: 0.4621 - val_accuracy: 0.7364
Epoch 107/200
175341/175341 [==============================] - 307s 2ms/step - loss: 0.2668 - accuracy: 0.8768 - val_loss: 0.4526 - val_accuracy: 0.7348
Epoch 108/200
175341/175341 [==============================] - 307s 2ms/step - loss: 0.2652 - accuracy: 0.8770 - val_loss: 0.4528 - val_accuracy: 0.7350
Epoch 109/200
175341/175341 [==============================] - 311s 2ms/step - loss: 0.2661 - accuracy: 0.8777 - val_loss: 0.4681 - val_accuracy: 0.7355
Epoch 110/200
175341/175341 [==============================] - 308s 2ms/step - loss: 0.2648 - accuracy: 0.8774 - val_loss: 0.4806 - val_accuracy: 0.7345
Epoch 111/200
175341/175341 [==============================] - 308s 2ms/step - loss: 0.2655 - accuracy: 0.8778 - val_loss: 0.4712 - val_accuracy: 0.7346
Epoch 112/200
175341/175341 [==============================] - 308s 2ms/step - loss: 0.2650 - ac

175341/175341 [==============================] - 308s 2ms/step - loss: 0.2592 - accuracy: 0.8796 - val_loss: 0.5553 - val_accuracy: 0.7336
Epoch 160/200
175341/175341 [==============================] - 310s 2ms/step - loss: 0.2585 - accuracy: 0.8790 - val_loss: 0.5318 - val_accuracy: 0.7420
Epoch 161/200
175341/175341 [==============================] - 308s 2ms/step - loss: 0.2591 - accuracy: 0.8792 - val_loss: 0.5627 - val_accuracy: 0.7302
Epoch 162/200
175341/175341 [==============================] - 308s 2ms/step - loss: 0.2576 - accuracy: 0.8793 - val_loss: 0.5423 - val_accuracy: 0.7349
Epoch 163/200
175341/175341 [==============================] - 309s 2ms/step - loss: 0.2575 - accuracy: 0.8802 - val_loss: 0.5717 - val_accuracy: 0.7352
Epoch 164/200
175341/175341 [==============================] - 310s 2ms/step - loss: 0.2572 - accuracy: 0.8799 - val_loss: 0.5568 - val_accuracy: 0.7389
Epoch 165/200
175341/175341 [==============================] - 309s 2ms/step - loss: 0.2574 - ac

In [7]:
from sklearn.metrics import accuracy_score,classification_report
predictions = model.predict([x,delta1])
predictions = [0 if i<0.5 else 1 for i in predictions]
print("Accuracy: ",accuracy_score(y,predictions))
from sklearn.metrics import confusion_matrix

print("Classification Report: \n",classification_report(y,predictions))
conmat = np.array(confusion_matrix(y, predictions, labels=[0,1]))
confusion = pd.DataFrame(conmat, index=['negative', 'positive'],
                         columns=['predicted_negative','predicted_positive'])
print (confusion)

Accuracy:  0.7430768109605014
Classification Report: 
               precision    recall  f1-score   support

           0       0.93      0.46      0.62     37000
           1       0.69      0.97      0.81     45332

    accuracy                           0.74     82332
   macro avg       0.81      0.72      0.71     82332
weighted avg       0.80      0.74      0.72     82332

          predicted_negative  predicted_positive
negative               17073               19927
positive                1226               44106
